In [1]:
import os
import sys
import time

#import ariadne.graph_net.model
import gin
import numpy as np
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from IPython.core.display import clear_output, display

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

/tmp/ipykernel_265068/1187170015.py:13: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output, display
/tmp/ipykernel_265068/1187170015.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output, display


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import logging
logging.getLogger().setLevel(logging.DEBUG)
    
from eval.event_evaluation import EventEvaluator
from ariadne_v2.transformations import Compose, DropShort, DropTracksWithHoles, DropSpinningTracks, BakeStationValues, ConstraintsNormalize, FixStationsBMN, ToCylindrical
from ariadne.utils.model import get_checkpoint_path, weights_update

In [4]:
from ariadne.utils.timer import CudaTimer

In [5]:
import scripts.clean_cache

#to clean cache if needed
#scripts.clean_cache.clean_jit_cache('20w')

## Tracknet

In [6]:
import torch
#torch.set_num_threads(1)
#torch.autograd.set_detect_anomaly(False)

import faiss
import onnxruntime as ort
#ort.set_default_logger_severity(0)

NUM_POINTS_TO_SEARCH = 1

from ariadne_v2.inference import IModelLoader
from ariadne.tracknet_v2.model import TrackNETv2
class TrackNetModelLoader(IModelLoader):    
    def __call__(self):
        
        tracknet_ckpt_path_dict = {'model_dir': '/lustre/home/user/d/drusov/ariadne_spd/ariadne/lightning_logs/TrackNETMissingHits',
                                   'version': 'version_19', 'checkpoint': 'latest'}# version 10
                                   #'version': 'OnlyRNN_alpha0.01_unbalanced', 'checkpoint': 'latest'}
        path_to_tracknet_ckpt = get_checkpoint_path(**tracknet_ckpt_path_dict)

        model_tr = weights_update(model=TrackNETInference(),
                       checkpoint=torch.load(path_to_tracknet_ckpt, map_location=torch.device(DEVICE)))
        model_tr.eval()
        model_tr = model_tr.to(DEVICE)
        for param in model_tr.parameters():
            param.requires_grad = False
        
        model_tr = torch.jit.script(model_tr).to(DEVICE)
        model_tr = torch.jit.optimize_for_inference(model_tr)
        """
        path_to_tracknet_ckpt = 'torchscript_model.pt'
        model_tr = torch.jit.load(path_to_tracknet_ckpt)
        model_tr.eval()
        """
        """
        torch.onnx.export(model_tr, 
                          dummy_tracks, 
                          "tracknetv3.onnx",
                          export_params=True,
                          opset_version=11,
                          do_constant_folding=True,  # whether to execute constant folding for optimization
                          input_names = ['inputs'],   # the model's input names
                          output_names = ['output'], # the model's output names
                          dynamic_axes={'inputs' : {0 : 'batch_size'},    # variable length axes
                                        'output' : {0 : 'batch_size'}})
        
        so = ort.SessionOptions()
        so.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
        providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]
        providers = providers if DEVICE == 'cuda' else providers[1:]
        model_tr = ort.InferenceSession('tracknetv3.onnx', sess_options=so, providers=providers)
        """
        model_hash = {
            "tracknet_ckpt_path_dict":path_to_tracknet_ckpt,
            'gin':gin.config_str(), 
            'model': '%r' % model_tr,
            'NUM_POINTS_TO_SEARCH':NUM_POINTS_TO_SEARCH
        }
        return model_hash, [model_tr]

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)
import ariadne.transformations as trn

tracknet_transformer = trn.Compose([
    #trn.DropShort(num_stations=4),
    #trn.DropTracksWithHoles(),
    #trn.DropSpinningTracks(),
    #trn.BakeStationValues(values=z_values),
    #trn.ConstraintsNormalize(constraints=constraints),
    #trn.ToCylindrical()
    ]
)

def preprocess_one_event(event_df):
    try:
        tracknet_data = tracknet_transformer(event_df)
    except AssertionError as err:
        #print("ASS error %r" % err)
        return None
    numpy_data = parse_df(tracknet_data)
    return ((numpy_data, tracknet_data), 1)

times = []

res_1 = None

from ariadne.graph_net.dataset import collate_fn

def run_tracknet_eval(result_from_prev_step, model, n_neighbours=NUM_POINTS_TO_SEARCH):
    numpy_data, event_df = result_from_prev_step[0]
    preds = run_model(numpy_data, model)
        
    #ev_first, ev_drop1, ev_drop2, ev_model1, ev_model2, ev_search2, ev_filter2, ev_prolong2, ev_last, ev_dropd_1, ev_dropd_2 = cuda_events
    #torch.cuda.synchronize()
    #times['total'].append(ev_first.elapsed_time(ev_last))
    #try:
    #    times['drop'].append(ev_drop1.elapsed_time(ev_drop2))
    #except:
    #    pass    
    #try:
    #    times['dropd'].append(ev_dropd_1.elapsed_time(ev_dropd_2))
    #except:
    #    pass
    #for i in range(34):
    #    try:
    #        times['model'].append(ev_model1[i].elapsed_time(ev_model2[i]))
    #    except:
    #        break
    #    times['search'].append(ev_model2[i].elapsed_time(ev_search2[i]))
    #    times['filter'].append(ev_search2[i].elapsed_time(ev_filter2[i]))
    #    times['prolong'].append(ev_filter2[i].elapsed_time(ev_prolong2[i]))
    
    result = build_df(preds, event_df)
    for col in [f'hit_id_{i}' for i in range(N_STATIONS)]:
        result[col] = result[col].astype(int)
    #res_1.append(result)
    return result

@CudaTimer.timeit(name='event')
def run_model(numpy_data, model):
    with torch.no_grad():
        inputs = setup(*numpy_data)
        preds = model[0](*inputs)
    return preds

cuda


In [7]:
N_STATIONS = 35

In [8]:
def get_seeds_only_first_station(df, columns=['x','y','z']):
    real = df
    temp1 = real[real.station == 0]
    st0_hits = temp1[columns].values
    
    seeds = torch.zeros((len(temp1), N_STATIONS, 3), dtype=torch.float32, device=DEVICE)
    #seeds = np.zeros((len(temp1), N_STATIONS, 3))
    seeds[:, 0, :] = torch.from_numpy(st0_hits)
    return seeds

def get_first_station_hits(df, columns=['x','y','z']):
    real = df
    temp1 = real[real.station == 0]
    st0_hits = temp1[columns].values
    return st0_hits

def build_hits(target_df, cols = ['x', 'y', 'z']):
    cont = torch.from_numpy(target_df[cols].values).contiguous().to(torch.float32).to(DEVICE)
    return cont

def parse_df(df, cols = ['x', 'y', 'z']):
    first_hits = get_first_station_hits(df)
    first_indexes = df[df.station == 0].index_old.values
    
    global_indexes = df.index_old.values
    global_hits = df[cols].values.astype('float32')
    
    #hits_by_station = []
    #indexes_by_station = []
    #for station in range(N_STATIONS):
    #    station_df = df.query('station == @station')
    #    hits_by_station.append(station_df[['x', 'y', 'z']].values.astype('float32'))
    #    indexes_by_station.append(station_df.index_old.values)
    return first_hits, first_indexes, global_hits, global_indexes
    #return first_hits, first_indexes, hits_by_station, indexes_by_station

#@CudaTimer.timeit(name='setup')
def setup(first_hits, first_indexes, global_hits, global_indexes):
#def setup(first_hits, first_indexes, hits_by_station, indexes_by_station):
    chunk_data_x = torch.zeros((len(first_hits), N_STATIONS, 4), dtype=torch.float32, device=DEVICE)
    chunk_data_x[:, 0, :3] = torch.from_numpy(first_hits)
    #r_axis = torch.sqrt(chunk_data_x[:, 0, 0]**2 + chunk_data_x[:, 0, 1]**2) + 0.02
    chunk_data_x[:, :, 3] = 1.#r_axis
    
    chunk_data_len = torch.tensor(np.full(len(chunk_data_x), 1), dtype=torch.int64)
    cand_mask = torch.ones(len(chunk_data_x), dtype=torch.int64, device=DEVICE)
    hits_indexes_global = torch.full((len(chunk_data_x), 35), -1, dtype=torch.int64, device=DEVICE)
    hits_indexes_global[:, 0] = torch.from_numpy(first_indexes)
    
    global_hits = torch.from_numpy(global_hits).to(DEVICE)
    global_indexes = torch.from_numpy(global_indexes).to(DEVICE)
    
    #hits_by_station = [torch.from_numpy(hits).to(DEVICE) for hits in hits_by_station]
    #indexes_by_station = [torch.from_numpy(indexes).to(DEVICE) for indexes in indexes_by_station]
    return chunk_data_x, cand_mask, hits_indexes_global, global_indexes, global_hits
    #return chunk_data_x, cand_mask, hits_indexes_global, hits_by_station, indexes_by_station

#@torch.jit.script
def drop_duplicated(cand_mask, chunk_data_x, hits_indexes_global):
    duplicated = hits_indexes_global[:, 0] != hits_indexes_global[:, 1]
    chunk_data_x = chunk_data_x[duplicated].contiguous()
    hits_indexes_global = hits_indexes_global[duplicated]
    cand_mask = cand_mask[duplicated]
    return cand_mask, chunk_data_x, hits_indexes_global

def build_df(indexes, event_df):
    #preds, indexes = preds
    new_columns = [*[f'hit_id_{i}' for i in range(N_STATIONS)], 'event', 'track_pred']
    if len(indexes) == 0:
        return pd.DataFrame(columns=new_columns).astype(float)
    indexes = indexes.cpu().numpy()
    #res_1.append(preds)
    curr_event = event_df.event.values[0]
    
    """
    big_index = build_index(event_df)
    event_idxs = np.array(event_df.index_old)
    tracks_list = []
    max_index = event_df.index.max()
    """
    
    #res_df = pd.DataFrame(columns=[f'hit_id_{i}' for i in range(N_STATIONS)])
    #res_df = pd.DataFrame(data=np.full_like(indexes, -1), columns=[f'hit_id_{i}' for i in range(N_STATIONS)])
    res_df = pd.DataFrame(data=indexes, columns=[f'hit_id_{i}' for i in range(N_STATIONS)])
    res_df['event'] = curr_event
    res_df['track_pred'] = True
    res_df['track_pred'] = res_df['track_pred'].astype(bool)
    #res_1.append(res_df.copy())
    
    #event_df.set_index(event_df.index_old, inplace=True)
    #for idx, track_hits in enumerate(indexes):
    #    track_hits = track_hits[track_hits > -1]
    #    stations, indices = np.unique(event_df.loc[track_hits].station.values, return_index=True)
    #    track_hits = track_hits[indices]
    #    try:
    #        res_df.loc[idx, [f'hit_id_{st}' for st in stations]] = track_hits
    #    except:
    #        res_df.drop(idx, inplace=True)
    #res_df = res_df.fillna(-1)
    """
    res_df = pd.DataFrame({'event': [curr_event] * len(preds),
                           'track_pred': np.ones(len(preds))#.astype(bool),
                          })
    for i in range(N_STATIONS):
        res_df[f'hit_id_{i}'] = indexes[:, i].numpy().astype(int)
    #res_df.track_pred = res_df.track_pred.astype('bool')
    #print(res_df)"""
    #res_1.append(res_df.copy())
    return res_df

COLS = ['x', 'y', 'z']

In [9]:
from typing import List

class TrackNETInference(torch.nn.Module):
    def __init__(self):
        super().__init__()
        input_features = 4
        hidden_features = 32
        output_features = 3
        batch_first = True
        self.rnn = torch.nn.GRU(
            input_size=input_features,
            hidden_size=hidden_features,
            num_layers=2,
            batch_first=batch_first
        )
        
        self.coords_1 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, output_features)
        )
        self.radius_1 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, 1),
            torch.nn.Softplus()
        )
        self.coords_2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, output_features)
        )
        self.radius_2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, 1),
            torch.nn.Softplus()
        )
        
    #def forward(self, chunk_data_x, cand_mask, hits_indexes_global, hits_by_station: List[torch.Tensor], indexes_by_station: List[torch.Tensor]):
    #@CudaTimer.timeit(name='event')
    def forward(self, chunk_data_x, cand_mask, hits_indexes_global, global_indexes, global_hits):
        # type: (Tensor, Tensor, Tensor, Tensor, Tensor) -> Tensor
        #timer.start('event')
        max_n_stations = 35

        for stations_gone in range(1, max_n_stations):
            #if stations_gone == 2:
            #    ev_dropd_1.record(stream)
            #timer.start('drop_duplicated')
            cand_mask, chunk_data_x, hits_indexes_global = self.drop_duplicated(cand_mask, chunk_data_x, hits_indexes_global, stations_gone)
            #timer.end('drop_duplicated')
            #    ev_dropd_2.record(stream)
            
            if stations_gone == 4:
                #timer.start('drop_short')
                cand_mask, chunk_data_x, hits_indexes_global = self.drop_short(cand_mask, chunk_data_x, hits_indexes_global)
                #timer.end('drop_short')
            #if len(hits_by_station[stations_gone]) == 0:
            #    break

            #timer.start('exec_model')
            prediction = self.exec_model(chunk_data_x, stations_gone)
            #timer.end('exec_model')
            #timer.start('search_torchdist')
            distances, nearest_hits, hits_indexes = self.search_torchdist(prediction[:, :4], global_hits, global_indexes)
            #timer.end('search_torchdist')
            #distances, nearest_hits, hits_indexes = self.search_torchdist(single_prediction, hits_by_station[stations_gone], indexes_by_station[stations_gone])
            #timer.start('filter')
            in_ellipse_mask = self.filter_hits_by_distance(prediction, distances)
            #timer.end('filter')
            
            not_ellipse_mask = ~in_ellipse_mask.to(torch.bool)
            #timer.start('second_search')
            distances2, nearest_hits2, hits_indexes2 = self.search_torchdist(prediction[not_ellipse_mask, 4:], global_hits, global_indexes)
            #timer.end('second_search')
            #timer.start('second_filter')
            in_ellipse_mask2 = self.filter_hits_by_distance(prediction[not_ellipse_mask, 4:], distances2)
            nearest_hits[not_ellipse_mask] = nearest_hits2
            hits_indexes[not_ellipse_mask] = hits_indexes2
            in_ellipse_mask[not_ellipse_mask] += in_ellipse_mask2
            #timer.end('second_filter')
            #print(in_ellipse_mask2)
            #timer.start('prolong')
            chunk_data_x, hits_indexes_global, cand_mask = self.prolong(in_ellipse_mask, chunk_data_x, stations_gone, nearest_hits, hits_indexes_global, hits_indexes, cand_mask)
            #timer.end('prolong')
        #timer.end('event')
        last_mask = hits_indexes_global[:, -2] == hits_indexes_global[:, -1]
        hits_indexes_global[last_mask, -1] = -1
        #res_1.append(chunk_data_x)
        return hits_indexes_global
    
    @CudaTimer.timeit(name='exec_model')
    def exec_model(self, inputs, stations_gone: int):
        x = inputs
        x, _ = self.rnn(x)
        coords_1 = self.coords_1(x)
        radius_1 = self.radius_1(x)
        coords_2 = self.coords_2(x)
        radius_2 = self.radius_2(x)
        outputs = torch.cat([coords_1, radius_1, coords_2, radius_2], dim=-1)
        #outputs = torch.cat([coords_1, radius_1], dim=-1)
        single_prediction = outputs[:, stations_gone - 1]
        #if printing:
        #    print(outputs)
        return single_prediction
    
    @CudaTimer.timeit(name='drop_short')
    def drop_short(self, cand_mask, chunk_data_x, hits_indexes_global):
        mask = cand_mask.to(torch.bool)
        chunk_data_x = chunk_data_x[mask].contiguous()
        hits_indexes_global = hits_indexes_global[mask]
        cand_mask = cand_mask[mask]
        return cand_mask, chunk_data_x, hits_indexes_global
    
    @CudaTimer.timeit(name='drop_duplicated')
    def drop_duplicated(self, cand_mask, chunk_data_x, hits_indexes_global, stations_gone: int):
        duplicated = hits_indexes_global[:, stations_gone - 2] != hits_indexes_global[:, stations_gone - 1]
        chunk_data_x = chunk_data_x[duplicated].contiguous()
        hits_indexes_global = hits_indexes_global[duplicated]
        cand_mask = cand_mask[duplicated]
        return cand_mask, chunk_data_x, hits_indexes_global
    
    @CudaTimer.timeit(name='search_torchdist')
    def search_torchdist(self, centers, global_hits, global_indexes):
        centers = centers[:, :3]
        D, I = torch.min(torch.cdist(centers, global_hits, compute_mode='use_mm_for_euclid_dist'), dim=1)
        hits_indexes = global_indexes[I]
        nearest_hits = global_hits[I]
        return D, nearest_hits, hits_indexes
    
    @CudaTimer.timeit(name='filter')
    def filter_hits_by_distance(self, ellipses, distances):
        return (distances.flatten() < ellipses[:, 3]).int()
    
    @CudaTimer.timeit(name='prolong')
    def prolong(self, in_ellipse_mask, chunk_data_x, stations_gone: int, nearest_hits, hits_indexes_global, hits_indexes, cand_mask):
        cand_mask = in_ellipse_mask * cand_mask
        chunk_data_x[:, stations_gone, :3] = nearest_hits * cand_mask.reshape(-1, 1)
        r_axis = torch.sqrt(chunk_data_x[:, :, 0]**2 + chunk_data_x[:, :, 1]**2) + 0.02
        chunk_data_x[:, :, 3] = r_axis
        hits_indexes_global[:, stations_gone] = hits_indexes * cand_mask + cand_mask - 1
        return chunk_data_x, hits_indexes_global, cand_mask

In [10]:
class TrackNETInference(torch.nn.Module):
    def __init__(self):
        super().__init__()
        input_features = 4
        hidden_features = 32
        output_features = 3
        batch_first = True
        self.rnn = torch.nn.GRU(
            input_size=input_features,
            hidden_size=hidden_features,
            num_layers=2,
            batch_first=batch_first
        )
        
        self.coords_1 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, output_features)
        )
        self.radius_1 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, 1),
            torch.nn.Softplus()
        )
        self.coords_2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, output_features)
        )
        self.radius_2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_features, 1),
            torch.nn.Softplus()
        )
        
    #@CudaTimer.timeit(name='event')
    def forward(self, chunk_data_x, cand_mask, hits_indexes_global, global_indexes, global_hits):
        # type: (Tensor, Tensor, Tensor, Tensor, Tensor) -> Tensor
        max_n_stations = 35

        for stations_gone in range(1, max_n_stations):
            
            if stations_gone == 4:
                cand_mask, chunk_data_x, hits_indexes_global = self.drop_short(cand_mask, chunk_data_x, hits_indexes_global)
            prediction = self.exec_model(chunk_data_x, stations_gone)
            
            first_dists, second_dists, first_nearest_hits, second_nearest_hits, first_hits_indexes, second_hits_indexes = self.search_both(prediction, global_hits, global_indexes)
            
            in_ellipse_mask1 = self.filter_hits_by_distance(prediction, first_dists)
            in_ellipse_mask2 = self.filter_hits_by_distance(prediction[:, 4:], second_dists)
            
            #second_nearest_hits = prediction[:, :3]
            #second_hits_indexes[:] = -1
            
            chunk_data_x, hits_indexes_global, cand_mask = self.prolong(in_ellipse_mask1, in_ellipse_mask2, chunk_data_x, stations_gone, first_nearest_hits, second_nearest_hits, hits_indexes_global, first_hits_indexes, second_hits_indexes, cand_mask)
            
            cand_mask, chunk_data_x, hits_indexes_global = self.drop_duplicated(cand_mask, chunk_data_x, hits_indexes_global, stations_gone)
            
        #last_mask = hits_indexes_global[:, -2] == hits_indexes_global[:, -1]
        #hits_indexes_global[last_mask, -1] = -1
        #res_1.append(chunk_data_x)
        #res_1.append(hits_indexes_global)
        return hits_indexes_global
    
    #@CudaTimer.timeit(name='exec_model')
    def exec_model(self, inputs, stations_gone: int):
        x = inputs
        x, _ = self.rnn(x)
        coords_1 = self.coords_1(x)
        radius_1 = self.radius_1(x)
        coords_2 = self.coords_2(x)
        radius_2 = self.radius_2(x)
        outputs = torch.cat([coords_1, radius_1, coords_2, radius_2], dim=-1)
        #outputs = torch.cat([coords_1, radius_1], dim=-1)
        single_prediction = outputs[:, stations_gone - 1]
        #if printing:
        #    print(outputs)
        return single_prediction
    
    #@CudaTimer.timeit(name='drop_short')
    def drop_short(self, cand_mask, chunk_data_x, hits_indexes_global):
        mask = cand_mask.to(torch.bool)
        chunk_data_x = chunk_data_x[mask].contiguous()
        hits_indexes_global = hits_indexes_global[mask]
        cand_mask = cand_mask[mask]
        return cand_mask, chunk_data_x, hits_indexes_global
    
    #@CudaTimer.timeit(name='drop_duplicated')
    def drop_duplicated(self, cand_mask, chunk_data_x, hits_indexes_global, stations_gone: int):
        duplicated = torch.logical_or(hits_indexes_global[:, stations_gone - 2] != hits_indexes_global[:, stations_gone - 1], hits_indexes_global[:, stations_gone - 1] == -1)
        chunk_data_x = chunk_data_x[duplicated].contiguous()
        hits_indexes_global = hits_indexes_global[duplicated]
        cand_mask = cand_mask[duplicated]
        return cand_mask, chunk_data_x, hits_indexes_global
    
    #@CudaTimer.timeit(name='search_torchdist')
    def search_torchdist(self, centers, global_hits, global_indexes):
        centers = centers[:, :3]
        D, I = torch.min(torch.cdist(centers, global_hits, compute_mode='use_mm_for_euclid_dist'), dim=1)
        hits_indexes = global_indexes[I]
        nearest_hits = global_hits[I]
        return D, nearest_hits, hits_indexes
    
    #@CudaTimer.timeit(name='filter')
    def filter_hits_by_distance(self, ellipses, distances):
        return (distances.flatten() < ellipses[:, 3]).int()

    #@CudaTimer.timeit(name='prolong')
    def prolong(self, in_ellipse_mask1, in_ellipse_mask2, chunk_data_x, stations_gone: int, first_nearest_hits, second_nearest_hits, hits_indexes_global, first_hits_indexes, second_hits_indexes, cand_mask):
        not_ellipse_mask = ~in_ellipse_mask1.to(torch.bool)
        first_nearest_hits[not_ellipse_mask] = 0.#second_nearest_hits[not_ellipse_mask]
        first_hits_indexes[not_ellipse_mask] = -1#second_hits_indexes[not_ellipse_mask]
        in_ellipse_mask1[not_ellipse_mask] = in_ellipse_mask2[not_ellipse_mask]
        
        cand_mask = in_ellipse_mask1 * cand_mask
        chunk_data_x[:, stations_gone, :3] = first_nearest_hits * cand_mask.reshape(-1, 1)
        chunk_data_x[not_ellipse_mask, stations_gone] = 0.
        #r_axis = torch.sqrt(chunk_data_x[:, :, 0]**2 + chunk_data_x[:, :, 1]**2) + 0.02
        #chunk_data_x[:, :, 3] = r_axis
        hits_indexes_global[:, stations_gone] = first_hits_indexes * cand_mask + cand_mask - 1
        return chunk_data_x, hits_indexes_global, cand_mask
    
    #@CudaTimer.timeit(name='search_both')
    def search_both(self, centers, global_hits, global_indexes):
        flattened = centers.reshape((-1, 4))
        flattened = flattened[:, :3]
        D, I = torch.min(torch.cdist(flattened, global_hits, compute_mode='use_mm_for_euclid_dist'), dim=1)
        dist = D.flatten().reshape(-1, 2)
        indexes = I.reshape((-1, 2))
        first_indexes = indexes[:, 0]
        second_indexes = indexes[:, 1]
        
        first_hits_indexes = global_indexes[first_indexes]
        first_nearest_hits = global_hits[first_indexes]
        second_hits_indexes = global_indexes[second_indexes]
        second_nearest_hits = global_hits[second_indexes]
        first_dists = dist[:, 0]
        second_dists = dist[:, 1]
        return first_dists, second_dists, first_nearest_hits, second_nearest_hits, first_hits_indexes, second_hits_indexes


In [11]:
import ariadne

In [12]:
from ariadne_v2.transformations import AddVirtualPoints, DropEmptyFirstStation, ToCylindrical, DropFakes, DropOverPhi, CombineEvents

In [13]:
from tqdm import tqdm

In [14]:
STATION_COLUMNS = [f'hit_id_{i}' for i in range(N_STATIONS)]

In [15]:
columns = ['event', 'x', 'y', 'z', 'station', 'track', 'px', 'py', 'pz', 'vtxx', 'vtxy', 'vtxz']

In [16]:
parse_cfg = {
    'csv_params' : {
        "sep": '\t',
        "encoding": 'utf-8',
        "names": columns
    },
    'input_file_mask':f'/lustre/home/user/d/drusov/ariadne_spd/ariadne/output/spd_tracknet_train_raw/spdsim_test_missinghits01_fakesnsqr_200k.tsv',
    'events_quantity': ':'
}

evaluator = EventEvaluator(parse_cfg, global_transformer, N_STATIONS)
events = evaluator.prepare(model_loader=loaded_model)[0]

In [17]:
loaded_model = TrackNetModelLoader()

In [18]:
constraints = {'x': [-851., 851.], 'y': [-851., 851.], 'z': [-2386., 2386.]}

In [19]:
global_transformer = Compose([
    DropShort(num_stations=3),
    DropEmptyFirstStation(),
    #DropTracksWithHoles(),
    #DropSpinningTracks(),
    #ToCylindrical(),
    #BakeStationValues(r_values, col='r'),
    ConstraintsNormalize(constraints=constraints, columns=('x', 'y', 'z')),
    #DropOverPhi(),
    #DropFakes(),
    CombineEvents(combine_n=40),
]
)

In [20]:
gin.enter_interactive_mode()

In [21]:
printing = False

In [22]:
res_1 = []
res_2 = []
times = []

In [23]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [24]:
evaluator = EventEvaluator(parse_cfg, global_transformer, N_STATIONS)
events = evaluator.prepare(model_loader=loaded_model)[0]
all_results = evaluator.build_all_tracks()

# COLD START

model = evaluator.loaded_model_state[1][0]
numpy_data = np.load('cold_start.npy', allow_pickle=True)
inputs = setup(*numpy_data)
for i in range(5):
    start_time = time.time()
    preds = model(*inputs)
    torch.cuda.synchronize()
    print(time.time() - start_time)
#model.save("torchscript_model.pt")
CudaTimer.clear()
model_results = evaluator.run_model(preprocess_one_event, run_tracknet_eval)
start_time = time.time()
results_tracknet = evaluator.solve_results(model_results, all_results, match_percentage=1)
end_time = time.time()
print(f'solving took {end_time - start_time} seconds')

read entry 62215056bdd09505e7cbf81302bb5f21 hit
[prepare]: started processing a df spdsim_test_missinghits01_fakesnsqr_200k.tsv with 199733753 rows:
read entry 087222e21a242674ddebc928376a08ce hit
[prepare] finished
[prepare] loading your model(s)...
/lustre/home/user/d/drusov/ariadne_spd/ariadne/lightning_logs/TrackNETMissingHits/version_19/*.ckpt
[prepare] finished loading your model(s)...
[build_all_tracks] start
read entry d54bbf42e409b5e3805b0fb357d40aac hit
read entry d12d1296d5dd9b010b06131efb0bec74 hit
[build_all_tracks] cache hit, finish
2.166612148284912
9.613042116165161
0.06113481521606445
0.061582326889038086
0.060616493225097656
[run model] start


model run: processed: 4999: 100%|██████████| 5000/5000 [05:49<00:00, 14.29it/s]
[run model] cache miss, finish
[solve results] start
[solve results] finish
[solve results] final stats:
==========EVALUATION RESULTS==========
Total events evaluated: 5000
Total tracks evaluated: 982026
Track Efficiency (recall): 0.9333 
Track Puri

In [ ]:
model_results[0]

In [ ]:
times = CudaTimer.times

In [ ]:
times_np = np.array(times['event'])
print(f'{1000 / times_np.mean()}, *40 = {40000 / times_np.mean()}')

In [ ]:
times['event'] = np.array(times['event'])
times['setup'] = np.array(times['setup'])
times['drop_short'] = np.array(times['drop_short'])
times['exec_model'] = np.array(times['exec_model'])
times['filter'] = np.array(times['filter'])
times['search_both'] = np.array(times['search_both'])
times['prolong'] = np.array(times['prolong'])
times['drop_duplicated'] = np.array(times['drop_duplicated'])
#times['second_search'] = np.array(times['second_search'])
#times['second_filter'] = np.array(times['second_filter'])

for key in times.keys():
    print(f'{key} time: {times[key].sum():.2f} ms, mean {times[key].mean():.2f} ms, ratio {times[key].sum() / times["event"].sum():.2f}')
print()
#print(f'Total time: {times["event"].sum():.2f} ms, mean {times["event"].mean():.2f} ms, timeslices per sec {40000 / times["event"].mean():.2f}')

In [ ]:
len(res_1)

In [ ]:
res_1 = res_1[5:]

In [ ]:
res_1[13]

In [ ]:
events.query('event == 6 and track == 2')

In [ ]:
res_1[12][1]

In [ ]:
res_1[13][1]

In [ ]:
model = evaluator.loaded_model_state[1][0]

In [ ]:
printing = True

In [ ]:
model.exec_model(hits_13)

In [ ]:
results_tracknet[0].query('pred == 0')

In [ ]:
tracks = results_tracknet[0].copy()
tracks['len'] = N_STATIONS - np.argmax(np.fliplr(tracks[STATION_COLUMNS] >= 0), axis=1)
tracks['with_holes'] = tracks.len - (tracks[STATION_COLUMNS] >= 0).sum(axis=1)

In [ ]:
tracks['first_hole'] = 0
for st in range(34, -1, -1):
    st_col = f'hit_id_{st}'
    tracks['first_hole'][tracks[st_col] == -1] = st

In [ ]:
tracks.query('pred == 0')

In [ ]:
(tracks.with_holes == 0).sum() / len(tracks)

In [ ]:
for i in range(N_STATIONS - 1):
    if (((tracks[STATION_COLUMNS[i]] == -1) + (tracks[STATION_COLUMNS[i + 1]] == -1)) == 2).any():
        print(i)

In [ ]:
np.unique(tracks.with_holes, return_counts=True)[0], np.unique(tracks.with_holes, return_counts=True)[1] / len(tracks)

In [ ]:
tracks.query('pred == 0 and first_hole == 1')

In [ ]:
tracks.query('event_id == 19')

In [ ]:
tracks.query('pred != -1 and with_holes > 0').first_hole.hist(bins=33)
plt.title("First hole on ? station (all tracks)")
plt.show()

In [ ]:
tracks.query('pred == 0 and with_holes > 0').first_hole.hist(bins=33)
plt.title("First hole on ? station (not found tracks)")
plt.show()

In [ ]:
for i in tqdm(range(625)):
    os.mkdir(f'faiss_samples/event{i}')
    with open(f'faiss_samples/event{i}/index.npy', 'wb') as f:
        np.save(f, res_1[i])
    os.mkdir(f'faiss_samples/event{i}/samples')
    for j in range(34):
        with open(f'faiss_samples/event{i}/samples/query{j}.npy', 'wb') as f:
            np.save(f, res_2[i][j])

In [ ]:
import numpy as np
import faiss
from tqdm import tqdm

In [ ]:
indexes = []
queries = []
for i in tqdm(range(625)):
    index_np = np.load(f'faiss_samples/event{i}/index.npy')
    indexes.append(torch.from_numpy(index_np))
    queries.append([])
    for j in range(34):
        query = np.load(f'faiss_samples/event{i}/samples/query{j}.npy')
        queries[-1].append(torch.from_numpy(query))

In [ ]:
DEVICE = 'cuda'

In [ ]:
indexes_f = []
for index_np in tqdm(indexes):
    index = faiss.IndexFlatL2(3)
    if DEVICE == 'cuda':
        index = faiss.index_cpu_to_gpu(faiss_gpu, 0, index)
    index.add(index_np)
    indexes_f.append(index)

In [ ]:
times = []
event_times = []
for index, queries_curr in tqdm(zip(indexes_f, queries)):
    event_start = time.time()
    #index = index.to(DEVICE)
    queries_gpu = [query.to(DEVICE) for query in queries_curr]
    for query in queries_gpu:
        #start_time = time.time()
        pr.enable()
        dist, i = index.search(query, 1)
        pr.disable()
        #times.append(time.time() - start_time)
    event_times.append(time.time() - event_start)

In [ ]:
np.array(times).mean()

In [ ]:
np.array(event_times).mean()

In [ ]:
times = []
event_times = []
for index, queries_curr in tqdm(zip(indexes, queries)):
    event_start = time.time()
    index = index.to(DEVICE)
    queries_gpu = [query.to(DEVICE) for query in queries_curr]
    for query in queries_gpu:
        #start_time = time.time()
        pr.enable()
        dist, i = search_t(query, index)
        pr.disable()
        #times.append(time.time() - start_time)
    event_times.append(time.time() - event_start)

In [ ]:
np.array(times).mean()

In [ ]:
np.array(event_times).mean()

In [ ]:
indexes[0].shape, queries[0][0].shape

In [ ]:
a = indexes_f[0].search(queries[0][0], 1)
a

In [ ]:
b = torch.min(torch.cdist(queries[0][0], indexes[0]), dim=1)
b

In [ ]:
(a[1].flatten() == b[1]).all()

In [ ]:
(a[0].flatten() == b[0]**2)

In [ ]:
(b[0]**2)[0]

In [ ]:
a[0][0]

In [ ]:
res_1[0].shape

In [ ]:
total_time = 0
for ev_id in tqdm(range(625)):
    index = faiss.IndexFlatL2(3)
    index.add(res_1[ev_id])
    
    start_time = time.time()
    for i in range(34):
        dist, i = index.search(res_2[ev_id][i], 1)
    total_time += time.time() - start_time
total_time / 625    

In [ ]:
len(res_1)

In [ ]:
len(res_2)

In [ ]:
res_2[0]

In [ ]:
pr = profile.Profile()
pr.disable()

In [ ]:
pr.enable()
for pair in tqdm(res_1, smoothing=0):
    a = search_distance(pair[0].cpu(), pair[1], 1)
pr.disable()

In [ ]:
for pair in tqdm(res_1, smoothing=0):
    #pr.enable()
    a = pair[1].range_search(pair[0].cpu().contiguous(), 0.00000087)
    #pr.disable()

In [ ]:
pr.disable()

In [ ]:
pr.dump_stats('profile1.pstat')

In [ ]:
res_1[0]

In [ ]:
result = results_tracknet[0].copy()
for ev in sorted(result.event_id.unique()):
    n_ghosts = len(result.query('event_id == @ev and pred == -1'))
    if n_ghosts > 0:
        max_track = result.query('event_id == @ev').track.max()
        result.track[(result.event_id == ev) & (result.pred ==-1)] = list(range(max_track + 1, max_track + n_ghosts + 1))

In [ ]:
events_1 = events.set_index('index_old')

In [ ]:
hits = []
for i in tqdm(range(0, 25000, 100)):
    result_part = result.query('@i <= event_id < @i + 100')
    events_1_part = events_1.query('@i <= event < @i + 100')
    for track in result_part.itertuples():
        hits_ids = np.array(track[7:])
        hits_df = events_1_part.loc[hits_ids[hits_ids > 0]]
        hits_df.track = track.track
        hits_df['pred'] = track.pred
        hits.append(hits_df)

In [ ]:
def denorm_coord(coord, min_val, max_val):
   return ((coord + 1) * (max_val - min_val) / 2) + min_val

In [ ]:
hits_to_kf = pd.concat(hits)
hits_to_kf.x = denorm_coord(hits_to_kf.x, constraints['x'][0], constraints['x'][1])
hits_to_kf.y = denorm_coord(hits_to_kf.y, constraints['y'][0], constraints['y'][1])
hits_to_kf.z = denorm_coord(hits_to_kf.z, constraints['z'][0], constraints['z'][1])

In [ ]:
hits_to_kf.sort_values('event', inplace=True)

In [ ]:
hits_to_kf.to_csv('1.txt', columns=['event', 'x', 'y', 'z', 'station', 'track', 'pred', 'px', 'py', 'pz', 'vtxx', 'vtxy', 'vtxz'], sep="\t", index=False, header=False)

In [ ]:
hits_to_kf.query('1998 <= event < 2001')

In [ ]:
pxpypz = pd.read_csv('result.csv', names=['event_id', 'track', 'px', 'py', 'pz'])
pxpypz

In [ ]:
merged_df = pd.merge(pxpypz, result, how='inner',
                           on=['event_id', 'track'],
                           suffixes=[None, '_y'])
merged_df.drop(['px_y', 'py_y', 'pz_y'], axis=1, inplace=True)

merged_df['track'][merged_df.pred == -1] = -1

merged_df.pred = merged_df.pred.astype('int32')
merged_df.track = merged_df.track.astype('int32')
merged_df.event_id = merged_df.event_id.astype('int32')

merged_df.px = merged_df.px.astype('float32')
merged_df.py = merged_df.py.astype('float32')
merged_df.pz = merged_df.pz.astype('float32')

In [ ]:
new_tracks = []
ind_col = []
for i in tqdm(range(0, 25000, 100)):
    result_part = result.query('@i <= event_id < @i + 100')
    pxpypz_part = pxpypz.query('@i <= event_id < @i + 100')
    for track in result_part.itertuples():
        if track.pred == -1:
            pxpypz_part_track = pxpypz_part.query('event_id == @track.event_id and track == @track.track')
            if len(pxpypz_part_track) > 0:
                track_dict = {}
                ind_col.append(track.Index)
                #track['index'] = pxpypz_part_track.index[0]
                track_dict['px'] = pxpypz_part_track.px.iloc[0]
                track_dict['py'] = pxpypz_part_track.py.iloc[0]
                track_dict['pz'] = pxpypz_part_track.pz.iloc[0]
                new_tracks.append(track_dict)
new_tracks = pd.DataFrame(new_tracks)
new_tracks = new_tracks.astype(result[['px', 'py', 'pz']].dtypes)
new_tracks.set_index(np.array(ind_col), inplace=True)
new_tracks

In [ ]:
joined_res = result.join(new_tracks, rsuffix='_kf')
joined_res.px = joined_res.px.fillna(joined_res.px_kf)
joined_res.py = joined_res.py.fillna(joined_res.py_kf)
joined_res.pz = joined_res.pz.fillna(joined_res.pz_kf)
joined_res.drop(['px_kf', 'py_kf', 'pz_kf'], axis=1, inplace=True)
joined_res.track[joined_res.pred == -1] = -1
joined_res

In [ ]:
joined_res

In [ ]:
results_tracknet[0].isna().sum()

In [ ]:
joined_res.isna().sum()

In [ ]:
start_time = time.time()
results_tracknet = evaluator.solve_results(model_results, all_results, match_percentage=1)
end_time = time.time()
print(f'solving took {end_time - start_time} seconds')

In [ ]:
results_tracknet[0].fillna(0, inplace=True)

## Plots

In [ ]:
%matplotlib inline

In [ ]:
def draw_track(ax, x, y, z, label, track_num):
    if label == 1:
        col = 'green'#(0., 1.0, 0.000, 0.9)
    elif label == 0:
        col = (0, 0.0, 1, 0.9)
    elif label == -1:
        pass
        col = (1, 0.0, 0.000, 0.9)
    else:
        col = (0, 1, 0, 0.3)
    val_x = x#track[:,0]
    val_y = y#track[:,2]
    val_z = z#track[:,1# ]
    #if track_num != 22 and track_num != 1285:
    #    col = list(col)[:3] + [0]
    line, = ax.plot(val_x, val_y, zs=val_z, color=col, label=f'track_num={track_num}', picker=10)
    #ax.scatter(val_x, val_y, zs=val_z, color=col)
    return line


def draw_event(ev_id, events_df, projection='xz', draw_unproc=False, columns=['x', 'y', 'z']):
    fig = plt.figure(figsize=(12,12))
    ax = plt.subplot(111, projection="3d")
    fig.add_axes(ax)
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    if projection == 'xz':
        ax.view_init(azim=200, elev=10)
        ax.set_xlabel(columns[0])
    else:
        ax.view_init(azim=360, elev=0)
        ax.set_ylabel(columns[1])
    i = 0
    lines = []

    hits_real = events_df.query('event == @ev_id and track >= 0')
    ax.scatter(hits_real[columns[0]], hits_real[columns[1]], zs=hits_real[columns[2]], c='lightgreen', alpha=0.5)
    
    hits_fake = events_df.query('event == @ev_id and track == -1')[:100]
    ax.scatter(hits_fake[columns[0]], hits_fake[columns[1]], zs=hits_fake[columns[2]], c='red', alpha=0.5)

    for tr_id, df in events_df.query('event == @ev_id').groupby('track'):
        if tr_id != -1:
            lines.append(draw_track(ax, df[columns[0]], df[columns[1]], df[columns[2]], 1, tr_id))

    #leg = ax.legend(fancybox=True, shadow=True)
    #lined = {}  # Will map legend lines to original lines.
    #for legline, origline in zip(leg.get_lines(), lines):
    #    legline.set_picker(True)  # Enable picking on the legend line.
    #    lined[legline] = origline

    """def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)"""
    plt.savefig('spd_event.png', dpi=300)
    plt.show()

In [ ]:
draw_event(7, events)

In [ ]:
result = results_tracknet[0].copy()
result['len'] = (result[STATION_COLUMNS] < 0).values.argmax(axis=1)
result.len = result.len.replace(0, N_STATIONS)

for tr_len in sorted(list(result.len.unique())):
    cands = result.query('len == @tr_len')
    try:
        precision = len(cands.query('pred == 1')) / (len(cands.query('pred == -1')) + len(cands.query('pred == 1')))
    except:
        precision = 0
    try:
        recall = len(cands.query('pred == 1')) / (len(cands.query('pred == 0')) + len(cands.query('pred == 1')))
    except:
        recall = 0
    print(f'tracklen = {tr_len}, number = {len(cands.query("pred != -1"))}({len(cands.query("pred != -1"))/len(result.query("pred != -1"))}), precision {precision}, recall {recall}')

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
from scipy import stats

MODES = {
    "RECALL":1,
    "PRECISION":2
}
a = []
def get_diagram_arr_linspace(all_real_hits, found_hits, start, end, num, col, mode):
    spac = np.linspace(start, end, num=num)
    #print(num)
    step = (spac[1] - spac[0]) / 2
    arr = []
    spac_ret = []
    track_count_for_arr = []
    for i in range(len(spac)-1):
        beg = spac[i]
        end = spac[i+1]
        elems_real = all_real_hits[(all_real_hits[col] >= beg) & (all_real_hits[col] < end)]
        elems_pred = found_hits[(found_hits[col] >= beg) & (found_hits[col] < end)]
        #print(beg,end)
        #print(len(elems_pred), len(elems_real))
        if mode == MODES["RECALL"]:
            if elems_real.empty:
                #arr.append(1.)
                continue
            else:
                arr.append(len(elems_pred) / len(elems_real))
                track_count_for_arr.append(len(elems_real))
        elif mode == MODES["PRECISION"]:
            if elems_pred.empty:
                #arr.append(1.)
                continue
            else:
                reco_true = len(elems_pred[elems_pred.track != -1])
                arr.append(reco_true / len(elems_pred))
                track_count_for_arr.append(len(elems_real))
        else:
            raise NotImplemented

        spac_ret.append(spac[i])

    return np.array(arr), np.array(spac_ret), np.array(track_count_for_arr)


def get_diagram_for_boxplot(all_real_hits, found_hits, start, end, num, col, bin_array, int_mode, mode):
    y_vals, x_vals, counts_ = get_diagram_arr_linspace(all_real_hits, found_hits, start, end, num, col, mode)

    np_y_vals = np.array(y_vals)
    bin_array = bin_array


    res_x_array = []
    res_box_data = []
    mean_box_array = []
    track_counts_ed = []
    for i in range(len(bin_array)-1):
        beg = bin_array[i]
        end = bin_array[i+1]
        y_ind = np.where((x_vals>=beg) & (x_vals<end))
        y_vals_corr  = np_y_vals[y_ind]
        track_counts_corr = counts_[y_ind]

        y_vals_corr = y_vals_corr[~np.isnan(y_vals_corr)]
        if len(y_vals_corr) == 0:
            continue

        #print(beg, end, i, stats.describe(y_vals_corr))
        #print(y_vals_corr)
        #         if len(np.where(y_vals_corr < 1)) > 0:
        #             print(y_vals_corr)
        #             print(y_vals)
        #             print(beg,end)

        res_box_data.append(y_vals_corr)
        track_counts_ed.append(np.sum(track_counts_corr))

        delta = 0 if int_mode else (end-beg)/2
        res_x_array.append(beg + delta)
        mean_box_array.append(np.mean(y_vals_corr))

    return res_box_data, np.array(res_x_array), np.array(mean_box_array), np.array(track_counts_ed)
    #plt.boxplot(res_box_data, positions=bin_array)

def boxplot_style(bp):
    #for box in bp['boxes']:
    # change outline color
    #box.set( color='#7570b3', linewidth=2)
    # change fill color
    #box.set( facecolor = 'silver' )

    ## change color and linewidth of the whiskers
    #for whisker in bp['whiskers']:
    #    whisker.set(color='#7570b3', linewidth=2)
    #
    ### change color and linewidth of the caps
    #for cap in bp['caps']:
    #    cap.set(color='#7570b3', linewidth=2)
    #
    ### change color and linewidth of the medians
    #for median in bp['medians']:
    #    median.set(color='tab:red', linewidth=3, zorder=30)

    for median in bp['means']:
        median.set(color='tab:green', linewidth=2, ls='-', zorder=10)


##########################
###########################

from numpy import linalg as LA
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

def draw_for_col(ax, tracks_real, tracks_pred_true,
                 col, col_pretty, total_events, n_ticks=150,
                 n_avg_ticks=-1, x_ticks=8,
                 custom_title=None,
                 ticks_custom=True,
                 with_boxplot=False,
                 int_mode=False, save_disk=True,
                 custom_draw_funcs=[],
                 diagram_func=None,
                 color_ax_ticks=False,
                 subtitle=None,
                 model_name="NO_MODEL_NAME",
                 graph_idx=0, total_indices=-1, spacing=-1,
                 color_line='tab:orange', color_box='red', mode=-1
                 ):
    real_xticks_count = x_ticks

    color_ax_ticks = len(custom_draw_funcs) > 0 and color_ax_ticks
    n_avg_ticks = n_ticks // 5 if n_avg_ticks < 0 else n_avg_ticks

    delta = 1e-4 if not int_mode else 1

    start = tracks_real[tracks_real[col] > -np.inf][col].min()
    end = tracks_real[tracks_real[col] < np.inf][col].max()+delta

    initial, spac, counts_ = get_diagram_arr_linspace(tracks_real, tracks_pred_true, start, end, n_ticks, col, mode)

    maxX = int(end) if int_mode else end
    ax.set_xlabel(col_pretty)
    ax.plot(spac, initial, alpha=0.0, lw=1)

    if not int_mode and ticks_custom:
        ax.set_xticks(np.linspace(start, maxX, x_ticks))
    else:
        ax.locator_params(axis='x', nbins=x_ticks)

    if diagram_func is None:
        ax.set_yticks(np.round(np.linspace(0, 1, 11), decimals=2))
    x_ticks = ax.get_xticks()

    if with_boxplot:
        old_ticks = x_ticks
        delta_x = (x_ticks[1] - x_ticks[0])/2

        diagram_func = get_diagram_for_boxplot if diagram_func is None else diagram_func

        box_data, ticks_x, mean_data, counts_ed = diagram_func(tracks_real, tracks_pred_true, start, end,
                                                               n_ticks, col, x_ticks, int_mode, mode)
        #print(x_ticks)
        if total_indices > 0:

            width_tuned = delta_x * 2 / total_indices
            pos_tuned = ticks_x - delta_x
            spacings = width_tuned * 0.4
            width_real = width_tuned * 0.6

            ticks_x = pos_tuned + width_tuned * graph_idx + spacings/2 + width_real/2
            delta_x = width_real
        box_data_t = []
        ticks_x_t = []
        delta_x_t = []
        mean_data_t = []
        counts_ed_t = []

        for i, box in enumerate(box_data):
            if len(box) > 4:
                box_data_t.append(box)
                ticks_x_t.append(ticks_x[i])
                mean_data_t.append(mean_data[i])

                counts_ed_t.append(counts_ed[i])

        box_data = np.array(box_data_t)
        ticks_x = np.array(ticks_x_t)
        mean_data = np.array(mean_data_t)

        counts_ed = np.array(counts_ed_t)

        bp = ax.boxplot(box_data, positions=ticks_x,
                        manage_ticks=False, meanline=True, showmeans=True,
                        widths=delta_x,patch_artist=True, sym='',zorder=3,boxprops=dict(facecolor=color_box, alpha=0.3))
        boxplot_style(bp)
        ret_widths = delta_x
        # mean line
        xnew = np.linspace(ticks_x.min(), ticks_x.max(), 500)
        spl = make_interp_spline(ticks_x, mean_data, k=1)
        power_smooth = spl(xnew)
        ax.plot(xnew, power_smooth, ls='--', color=color_line, label=f'{model_name} mean track {subtitle}', lw=3, zorder=15)
        #ax.set_xticks(old_ticks)
        if graph_idx == 0:
            for i in range(len(ticks_x)):
                y_pos = 1.01
                x_pos = ticks_x[i] - delta_x/2
                ax.text(x_pos, y_pos, f"{counts_ed[i]}", zorder=80)

    if int_mode or not ticks_custom:
        ax.locator_params(axis='x', nbins=real_xticks_count)

    #if diagram_func is None:
    #ax.set_ylim((0.8, 1.05))
    ax.set_ylim((-0.05, 1.05))
    ticks = ax.get_xticks()
    step = ticks[1] - ticks[0]
    if graph_idx == total_indices-1:
        ax.set_xlim((np.min(ticks_x) - width_tuned-step, np.max(ticks_x)+width_real + step/2))

    for draw_f in custom_draw_funcs:
        draw_f(ax)

    return ret_widths

    ####plt.locator_params(axis='y', nbins=16)


    #plt.tight_layout()
    #plt.rcParams['savefig.facecolor']='white'
    #os.makedirs('../output', exist_ok=True)
    #plt.savefig('../output/new_img_track_eff_%s_ev%r_t%d.png'%(col, total_events, n_ticks), dpi=300)
    #plt.show()

def draw_for_raw_data(ax, data_x, data_y, data_y_err, color_box, color_line, col_widths, model_name, subtitle):
    data_x = np.array(data_x)
    data_y_init = np.array(data_y)
    dataep = data_y + np.array(data_y_err)
    dataem = data_y - np.array(data_y_err)

    data_y = np.expand_dims(data_y,axis=-1)
    dataep = np.expand_dims(dataep,axis=-1)
    dataem = np.expand_dims(dataem,axis=-1)

    data_y = np.concatenate((data_y, dataep, dataem), axis=1).T
    delta_x = (data_x[1]-data_x[0]) / 2

    width_tuned = delta_x * 2 / 3
    pos_tuned = data_x - delta_x
    spacings = width_tuned * 0.4
    width_real = width_tuned * 0.6

    ticks_x = pos_tuned + width_tuned * 2 + spacings/2 + width_real/2

    bp = ax.boxplot(data_y, positions=ticks_x,
                    manage_ticks=False, meanline=True, showmeans=True,
                    widths=col_widths, patch_artist=True, sym='',zorder=3,boxprops=dict(facecolor=color_box, alpha=0.3))
    boxplot_style(bp)
    xnew = np.linspace(ticks_x.min(), ticks_x.max(), 500)
    mean_data = data_y_init
    spl = make_interp_spline(ticks_x, mean_data, k=1)  # type: BSpline
    power_smooth = spl(xnew)

    ax.plot(xnew, power_smooth, ls='--', color=color_line, label=f'{model_name} mean track {subtitle}', lw=3, zorder=15)

    pass

def to_recall(all_tracks_df):
    tracks_real = all_tracks_df[all_tracks_df.pred != -1]
    tracks_pred_true = all_tracks_df[all_tracks_df.pred == 1]
    return tracks_real, tracks_pred_true, MODES["RECALL"], "efficiency"

def to_precision(all_tracks_df):
    tracks_real = all_tracks_df[all_tracks_df.track != -1]
    tracks_pred_true = all_tracks_df[(all_tracks_df.pred == 1) | (all_tracks_df.pred == -1)]
    return tracks_real, tracks_pred_true, MODES["PRECISION"], "purity"

def plot_model_results(model_names, model_results_arr, mode_func, funcs_for_kalman=None):

    fig = plt.figure(figsize=(14,16), dpi=80)
    ax1, ax2, ax3 = fig.subplots(nrows=3, ncols=1)
    #    ax1, ax2 = fig.subplots(nrows=1, ncols=2, sharey='all')

    total_models = len(model_names)
    if funcs_for_kalman:
        total_models+=1

    colormap = plt.cm.tab10 #nipy_spectral, Set1,Paired
    colors = [colormap(i) for i in np.linspace(0, 1, 8)]

    widths_for_ax = []
    tit = ""
    for idx, (model_name, model_results) in enumerate(zip(model_names, model_results_arr)):

        all_tracks_df = model_results[0]

        all_tracks_df['pt'] = LA.norm(all_tracks_df[['px','py']].values, axis=1)
        all_tracks_df['pt'] = all_tracks_df['pt'].astype('float32')
        all_tracks_df['cos_t'] = (all_tracks_df[['pz']].values/ LA.norm(all_tracks_df[['px','py','pz']].values, axis=1, keepdims=True))
        all_tracks_df['a_phi'] = np.arctan2(all_tracks_df[['px']].values, all_tracks_df[['py']].values)

        tracks_real, tracks_pred_true, mode, subtitle  = mode_func(all_tracks_df)

        color_box = colors[idx]
        color_line = colors[idx]
        n_events = model_results_arr[0][0].event_id.nunique()
        tit = f"Models track {subtitle} on {n_events} events"
        ax1.set_title(tit)
        ax1.set_ylabel(f'Track {subtitle}', fontsize=12)
        #ax2.set_ylabel(f'Track {subtitle}', fontsize=12)
        #ax3.set_ylabel(f'Track {subtitle}', fontsize=12)

        widths_ax = draw_for_col(ax1, tracks_real, tracks_pred_true, 'pt', '$pt$', n_events, 350,
                                 n_avg_ticks=48, x_ticks=20, ticks_custom=False, with_boxplot=True, model_name=model_name,
                                 graph_idx=idx, total_indices=total_models, subtitle=subtitle,
                                 color_line=color_line, color_box=color_box, mode=mode)

        widths_for_ax.append([widths_ax])


        widths_ax = draw_for_col(ax2, tracks_real, tracks_pred_true, 'a_phi', '$a_\phi$',n_events, 350,
                                 n_avg_ticks=48, x_ticks=20, with_boxplot=True, model_name=model_name,
                                 graph_idx=idx, total_indices=total_models, subtitle=subtitle,
                                 color_line=color_line, color_box=color_box, mode=mode)
        widths_for_ax[-1].append(widths_ax)
        #
        ##enable on big dataset
        widths_ax = draw_for_col(ax3, tracks_real, tracks_pred_true,'cos_t', '$cos_t$',n_events, 350,
                                 n_avg_ticks=48,x_ticks=20, ticks_custom=False, with_boxplot=True, model_name=model_name,
                                 graph_idx=idx, total_indices=total_models, subtitle=subtitle,
                                 color_line=color_line, color_box=color_box, mode=mode)
        widths_for_ax[-1].append(widths_ax)

    if funcs_for_kalman:
        for idx, ax in enumerate([ax1,ax2,ax3]):
            color_box = colors[-2]
            color_line = colors[-2]
            funcs_for_kalman[idx](ax, widths_for_ax[-1][idx], color_box, color_line)

    for ax in [ax1,ax2,ax3]:
        ax.grid()
        ax.legend(bbox_to_anchor=(1, -0.05))

    fig = plt.gcf()
    handles, labels = ax1.get_legend_handles_labels()
    #fig.legend(handles, labels, loc='lower right', bbox_to_anchor=(1, -0.05))
    #plt.title()
    plt.tight_layout()
    plt.rcParams['savefig.facecolor']='white'

    os.makedirs('../output', exist_ok=True)
    #plt.savefig('../output/%s.png'%(''.join([i if (ord(i) < 128) and (ord(i)>ord('a')) else '_' for i in tit])), dpi=300)
    #return all_tracks_df
    plt.show()
    pass

In [ ]:
plot_model_results(["TrackNetV3"],    #
                   [[joined_res, ]],
                   to_precision)

In [ ]:
plot_model_results(["TrackNetV3"],
                   [results_tracknet],
                   to_precision)

In [ ]:
plot_model_results(["TrackNetV3"],
                   [results_tracknet],
                   to_recall)